In [47]:
# Este ficheiro diz respeito ao vento na zona
from netCDF4 import Dataset, num2date

import numpy as np

import pandas as pd

from datetime import datetime, timedelta

In [48]:
#ler o ficheiro netcdf
file1 = r'C:\Users\35192\Desktop\CERSAT-GLO-REP_WIND_L4-OBS_FULL_TIME_SERIE_1634566865410.nc'

#ler a base de dados 
file2 = r'C:\Users\35192\Desktop\Tese\Criar Base de Dados\Base_Dados.csv'

In [49]:
data1 = Dataset(file1, mode='r')

In [50]:
#reproduzir as variáveis que estão contidas no ficheiro netcdf
variables = list(data1.variables)

print(variables)

['wind_speed_std', 'time', 'depth', 'latitude', 'longitude', 'wind_speed_rms', 'wind_speed']


In [51]:
data2 = pd.read_csv(file2)

#aceder aos dados da base de dados que contêm a variável 'time'
print(data1.variables['time'])

<class 'netCDF4._netCDF4.Variable'>
int32 time(time)
    long_name: time
    units: hours since 1900-01-01 00:00:00.0 00:00
    valid_min: 1051164.0
    valid_max: 1051164.0
    axis: T
    standard_name: time
    calendar: gregorian
    _CoordinateAxisType: Time
unlimited dimensions: 
current shape = (130,)
filling on, default _FillValue of -2147483647 used


In [58]:
# calendar: gregorian, units: hours since 1900-01-01 00:00:00.0 00:00

 

time1 = data1.variables['time'][:].data #saber de todas as datas

 

time_modif = num2date(time1,

                        units = 'hours since 1900-01-01 00:00:00.0 00:00',

                        calendar = 'gregorian') # como modificar de numérico para data gregorian

In [59]:
date1 = []

 

for i in range(len(time_modif)): # mudar para datetime

    b = time_modif[i]

    reg = datetime(b.year,b.month,b.day,b.hour)

    date1.append(reg)

In [60]:

latitude1 = data1.variables['latitude'][:].data # guardar todas as latitudes

longitude1 = data1.variables['longitude'][:].data # guardar todas as longitudes

 

latitude2 = data2.lat_d

longitude2 = data2.lon_d

In [61]:
# como as latitudes e longitude não coincidem convem ter um aproximador
def nearest_local(local,coordinates_data):
    distance_to_local = np.sqrt(np.square(coordinates_data-local).sum(axis=1))
    ind_local = np.argmin(distance_to_local)
    return ind_local, coordinates_data[ind_local]
 

In [62]:
local = [latitude2[0],longitude2[0]]

In [93]:
coordinates_data1 = np.array(np.meshgrid(latitude1,longitude1)).T.reshape(-1,2)

 

print(local)

print(nearest_local(local,coordinates_data1))

[-4.75, 11.783333333333]
(13319, array([-4.875, 11.875], dtype=float32))


[-4.75, 11.783333333333]

In [72]:
data1.variables['time'].units

'hours since 1900-01-01 00:00:00.0 00:00'

In [75]:
data1.variables['time'].units[12:22] #extrair as datas diretamente do netcdf file

'1900-01-01'

In [76]:
starting_date = data1.variables['time'].units[12:22]

In [86]:
ending_date = data1.variables['time'].units[12:16] + '-12-31'

'1900-12-31'

In [87]:
date_range = pd.date_range(start = starting_date, end = ending_date)

In [88]:
df = pd.DataFrame(0, columns = ['wind_speed'], index = date_range)

In [90]:
data1.variables['time']

<class 'netCDF4._netCDF4.Variable'>
int32 time(time)
    long_name: time
    units: hours since 1900-01-01 00:00:00.0 00:00
    valid_min: 1051164.0
    valid_max: 1051164.0
    axis: T
    standard_name: time
    calendar: gregorian
    _CoordinateAxisType: Time
unlimited dimensions: 
current shape = (130,)
filling on, default _FillValue of -2147483647 used

In [92]:
dt = np.arange(0,data1.variables['time'].size)

In [ ]:
for time_index in dt:
    df.iloc[time_index] = wind[time_index,